<a href="https://colab.research.google.com/github/makhlouf279/Capstone-Project---The-Battle-of-Neighborhoods/blob/main/The_Battle_of_Neighborhoods_week1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMB Capstone Project: The Battle of Neighborhoods**
OCTOBER, 24/2020





**Data Section**

1. For the above objective, we will be using open-data acquired from the Dubai
Statistics Center. The data is available in the form of an Excel sheet, which
will require a considerable amount of refinement. The data source is
accessible at below location:

Data Source: https://www.dsc.gov.ae/Report/DSC_SYB_2019_01%20_%2002.xlsx

- Description: This data set contains the required information. And we will use
this data set to explore various neighborhoods of Dubai city.
- https://en.wikipedia.org/wiki/Dubai

**2. Coffee shop in neighborhoods of Dubai city.**

- Data Source: Foursquare API
- Description: By using this API we will get all the venues in the neighborhoods.
We can filter these venues to get only Pizza Places. 

**Approach**

- Collect the Dubai city data
from https://www.dsc.gov.ae/Report/DSC_SYB_2019_01%20_%2002.xlsx.
- Using Foursquare API we will get all venues for each neighborhood.
- Filter out all venues which are coffee shop.
- Analyzing using Clustering (Specially K-Means):
1. Find the best value of K
2. Visualize the neighborhood with a number of coffee shop.
- Compare the Neighborhoods to Find the Best Place for Starting up a coffee
shop 






In [182]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup # library for web scrapping 
import geocoder
from geopy.geocoders import Nominatim
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [183]:
pip install  geocoder

**Step 1.1: Extract data**

In [185]:
# reading excel report from the source.

data_url = 'https://www.dsc.gov.ae/Report/DSC_SYB_2019_01%20_%2002.xlsx'
df_dubai = pd.read_excel(data_url)

# determining structure
df_dubai.shape

(247, 5)

**Step 1.1: Data Wrangling:**

Because the report has a considerable amount of header and footer data, we will be removing it.

In [186]:
df_dubai.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,عدد السكان المقدر حسب القطاع والمنطقة - إمارة ...,NaN,NaN,NaN,NaN
2,Number of Estimated Population by Sector and C...,NaN,NaN,NaN,NaN
3,` (2019),NaN,NaN,NaN,NaN
4,جـــدول ( 02 - 01 ) Table,NaN,NaN,NaN,NaN
5,رقم \nالمنطقة,القطاع والمنطقة,مجموع السكان\nTotal population,Sector & Community,Community Code
6,NaN,NaN,NaN,NaN,NaN
7,101,نخلة ديرة,2,NAKHLAT DEIRA,101
8,111,الكورنيش,1735,AL CORNICHE,111
9,112,الرأس,7460,AL RASS,112


In [187]:
# removing header information

df_dubai = df_dubai.iloc[7:]
df_dubai.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
7,101,نخلة ديرة,2,NAKHLAT DEIRA,101
8,111,الكورنيش,1735,AL CORNICHE,111
9,112,الرأس,7460,AL RASS,112
10,113,الضغاية,15899,AL DHAGAYA,113
11,114,البطين,2841,AL BUTEEN,114


In [188]:
# removing footer from the report

df_dubai = df_dubai[:-6]
df_dubai.tail()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
236,978,سيح شعيله,3,SAIH SHUA'ALAH,978
237,981,مقطره,804,MUGATRAH,981
238,987,الليان 1,10,AL LAYAN 1,987
239,988,الليان 2,0,AL LAYAN 2,988
240,991,حفير,0,HEFAIR,991


The report also contains additional columns which we do not require as they represent the same information in Arabic.

In [189]:
df_dubai = df_dubai[['Unnamed: 2', 'Unnamed: 3']]

Renaming column headers

In [190]:
df_dubai.rename(columns = {'Unnamed: 2':'population', 'Unnamed: 3':'community'}, inplace = True)

**Step 1.3: Data Wrangling Continues:**

If you look at the report, the communities are split in Sectors. These sectors are in the report as splitter rows, which we need to remove.

In [192]:
# Get the indexs of the rows which have text like 'Sector' in community column

sector_index = df_dubai[df_dubai['community'].isin(['Sector 1', 'Sector 2', 'Sector 3', 'Sector 4', 'Sector 5', 'Sector 6', 'Sector 7', 'Sector 8'])].index

# droping the rows based on found indeces
df_dubai.drop(sector_index, inplace = True)

df_dubai.shape

(226, 2)

Lets change the order of the columns in our dataframe

In [193]:
df_dubai = df_dubai[['community', 'population']]
df_dubai.head()

,community,population
7,NAKHLAT DEIRA,2
8,AL CORNICHE,1735
9,AL RASS,7460
10,AL DHAGAYA,15899
11,AL BUTEEN,2841


Lets sort the dataframe by population (descending)

In [194]:
df_dubai.sort_values(by = ['population'], inplace = True, ascending = False)
df_dubai.head(10)

,community,population
56,MUHAISANAH SECOND,196316
107,AL GOZE IND. SECOND,159978
153,JABAL ALI INDUSTRIAL FIRST,128975
163,WARSAN FIRST,106072
23,HOR AL ANZ,83187
147,JABAL ALI FIRST,75287
77,AL KARAMA,75066
152,DUBAI INVESTMENT PARK1,69956
20,AL MURQABAT,69771
51,MURDAF,64355



We will be extracting coordinates using GeoPy by leveraging Google Maps or some other data source provider. When I was scouting for the data, I noticed that in our report, the area names have a suffix like FIRST, SECOND, THIRD, ETC., While the same areas were marked with number 1, 2, 3 in Google Maps.

This means if I have to pass WARSAN FIRST to GeoPy, it won't find the coordinates. To solve this problem, we will replace the suffix with numerical values

In [195]:
df_dubai.replace('FIRST', '1', regex = True, inplace = True)
df_dubai.replace('SECOND', '2', regex = True, inplace = True)
df_dubai.replace('THIRD', '3', regex = True, inplace = True)
df_dubai.replace('FOURTH', '4', regex = True, inplace = True)
df_dubai.replace('FIFTH', '5', regex = True, inplace = True)
df_dubai.replace('SIXTH', '6', regex = True, inplace = True)
df_dubai.head(5)

,community,population
56,MUHAISANAH 2,196316
107,AL GOZE IND. 2,159978
153,JABAL ALI INDUSTRIAL 1,128975
163,WARSAN 1,106072
23,HOR AL ANZ,83187



Removing industrial areas from out list of communities as we are only intreseted in commercial+residential areas for our Coffee Shop

In [196]:
df_dubai = df_dubai[~df_dubai.community.str.contains('IND.')]
df_dubai.head()

,community,population
56,MUHAISANAH 2,196316
163,WARSAN 1,106072
23,HOR AL ANZ,83187
147,JABAL ALI 1,75287
77,AL KARAMA,75066



Some of the names of locality in this dataset were not as they are represented in map providers. For example, 'Al Quoz' is named as 'Al Goze.' This can cause inconsistency and may leave us excluding the populated areas from our analysis. Following are the naming corrections which we had to.

In [197]:
df_dubai.replace('JABAL ALI 1', 'JEBEL ALI', regex = True, inplace = True)
df_dubai.replace('AL KALIJ AL TEJARI', 'BUSINESS BAY', inplace = True)
df_dubai.replace('AL WAHEDA', 'AL WUHEIDA', inplace = True)
df_dubai.replace('AL THANYAH 3 (EMIRATE HILLS 2)', 'EMIRATES HILLS 2', inplace = True)
df_dubai.replace('NADD HESSA', 'DUBAI SILICON OASIS', inplace = True)
df_dubai.replace('AL THANYAH 1 (V. RABIE SAHRA\'A)', 'TECOM', inplace = True)
df_dubai.replace('MENA JABAL ALI', 'JEBEL ALI NORTH FREE ZONE', inplace = True)
df_dubai.replace('AL HEBIAH 4', 'DUBAI SPORTS CITY', inplace = True)
df_dubai.replace('UM SOUQAIM 2', 'UMM SUQEIM 2', inplace = True)
df_dubai.replace('UM SOUQAIM 1', 'UMM SUQUEIM 1', inplace = True)
df_dubai.replace('AL HEBIAH 1', 'MOTOR CITY', inplace = True)
df_dubai.replace('MURDAF', 'MIRDIF', regex = True, inplace = True)
df_dubai.replace('PARK1', 'PARK 1', regex = True, inplace = True)
df_dubai.replace('PARK2', 'PARK 2', regex = True, inplace = True)
df_dubai.replace('MURQABAT', 'MURAQABAT', regex = True, inplace = True)
df_dubai.replace('MARSA DUBAI (AL MINA AL SEYAHI) ', 'MARSA DUBAI', inplace = True)
df_dubai.replace('AL BADA', 'AL BADA\'A', regex = True, inplace = True)
df_dubai.replace('SUQ', 'SOUQ', regex = True, inplace = True)
df_dubai.replace('AL THANYAH 5 (EMIRATE HILLS 1) ', 'EMIRATES HILLS 1', inplace = True)
df_dubai.replace('AL THANYAH 4 (EMIRATE HILLS 3) ', 'EMIRATES HILLS 3', inplace = True)
df_dubai.replace('NAD AL HAMAR', 'NADD AL HAMAR', inplace = True)
df_dubai.replace('AL SOUQ AL KABEER', 'BUR DUBAI', inplace = True)
df_dubai.replace('AL BAESHAA 2', 'AL BARSHA 2', inplace = True)
df_dubai.replace('MADINAT DUBAI AL MELAHEYAH (AL MINA)', 'DUBAI MARITIME CITY', inplace = True)
df_dubai.replace('AL DHAGAYA', 'AL RAS', inplace = True)
df_dubai.replace('GOZE', 'QUOZ', regex = True, inplace = True)
df_dubai.replace('AL REGA', 'AL RIGGA', inplace = True)
df_dubai.replace('WADI AL SAFA 3', 'LIVING LEGENDS', inplace = True)
df_dubai.replace('AL HEBIAH 5', 'REMRAAM', inplace = True)
df_dubai.replace('AL SAFFA 1', 'AL SAFA 1', inplace = True)
df_dubai.replace('UM SOUQAIM 3', 'UMM SUQEIM 3', inplace = True)
df_dubai.replace('REGA AL BUTEEN', 'RIGGAT AL BUTEEN', inplace = True)
pd.set_option('display.max_rows', None)
df_dubai.replace('MUHAISANAH 4', 'MUHAISNAH 4', inplace = True)
df_dubai.replace('OUD AL MUTEEN 1', 'OUD AL MUTEENA 1', inplace = True)
df_dubai.replace('WADI AL SAFA 6 (ARABIAN RANCHES)', 'ARABIAN RANCHES', inplace = True)

Now that we have our desired dataframe, we will proceed to Stage 2 of our work.

**Stage 2: Coordinates:**

In stage 2, we will extract each community's coordinates and append it to our data frame.

To minimize the time required to extract such information, we will be obtaining the coordinates of the top 100 communities with the highest population.

**Step 2.1: Top 100**

In [198]:
# getting the top 200 communities based on population

df_communities = df_dubai.head(200)

**Step 2.2: GeoPy**

In [199]:
# defining function to get coordinates based on community name

def get_latitude_longitude(community_name):
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    #while(location is None):
    geolocator = Nominatim(user_agent="waqa5_ahm3d_capstone")
    location = geolocator.geocode('{}, Dubai, United Arab Emirates'.format(community_name))
    
    latitude = location.latitude
    longitude = location.longitude
    
    return latitude, longitude

Now time to loop through Top 100 communities and append their coordinates into dataframe

In [200]:

for i, row in df_communities.head(100).iterrows():
    community_name = row['community']
    
    #Function call
    try:
        lat, long = get_latitude_longitude(community_name)
        
        #Appending to dataframe
        df_communities.loc[i, 'latitude'] = lat
        df_communities.loc[i, 'longitude'] = long
    except:
        pass

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [201]:
#Dropping NaN entries from our dataset

df_communities.dropna(inplace = True)
df_communities.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,community,population,latitude,longitude
56,MUHAISANAH 2,196316,25.280555,55.410502
163,WARSAN 1,106072,25.163154,55.422077
23,HOR AL ANZ,83187,25.279548,55.341053
147,JEBEL ALI,75287,25.040996,55.133560
77,AL KARAMA,75066,25.238448,55.303458
152,DUBAI INVESTMENT PARK 1,69956,25.010873,55.165855
20,AL MURAQABAT,69771,25.265104,55.329721
51,MIRDIF,64355,25.220229,55.423000
43,AL NAHDA 2,61936,25.290592,55.376731
121,MARSA DUBAI,61047,25.087754,55.146172



As you can see from above, it requires allot of efforts to make your data usable as per your requirement.

I will be saving this dataset and will publish this on Kaggle for anyone in future looking for top 100 communities in Dubai along with their population.

In [202]:
print('The dataframe has {} communities.'.format(
        len(df_communities['community'].unique()),
        df_communities.shape[0]
    )
)

#Resetting index

df_communities.reset_index(drop=True, inplace=True)

The dataframe has 89 communities.



**Stage 3: Mapping:**

Let's take a look at Dubai and based on our dataset, lets see where all these communities are.

For mapping, we will be using Folium.

**Step 3.1: Get Dubai city coordinates**

In [203]:
#Using Nominatim, we will get latitude and longitude for Dubai city

dxb_address = 'Dubai, United Arab Emirates'

geolocator = Nominatim(user_agent="dxb_explorer")
location = geolocator.geocode(dxb_address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Dubai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dubai are 25.0750095, 55.18876088183319.


**Step 3.2: Mapping Dubai via Folium**

With Folium, we will map out Dubai and then place markers for each community we have in our dataframe

In [204]:
# create map of Dubai using latitude and longitude values
map_dubai = folium.Map(location = [latitude, longitude], zoom_start = 11)

# add markers to map
for lat, lng, label in zip(df_communities['latitude'], df_communities['longitude'], df_communities['community']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 7,
        popup = label,
        color = 'red',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_dubai)  
    
map_dubai